In [29]:
tri0 = location[location.place_id == location.place_id.unique()[0]]
tri1 = location[location.place_id == location.place_id.unique()[1]]
tri2 = location[location.place_id == location.place_id.unique()[2]]
# %matplotlib inline
# plt.scatter(tri0.x, tri0.y) # Mostly a single x, some scatter in y. Lots of outliers.

In [3]:
tri0_stats = tri0[['x', 'y', 'accuracy', 'time']].describe()

# Compute outliers (by boxplot)
# http://stackoverflow.com/questions/17725927/boxplots-in-matplotlib-markers-and-outliers
IQR = tri0_stats.iloc[6] - tri0_stats.iloc[4]
rng = tri0_stats.iloc[7] - tri0_stats.iloc[3]
tri0_stats = tri0_stats.append(rng, ignore_index=True)
tri0_stats = tri0_stats.append(IQR, ignore_index=True)
out_min = tri0_stats.iloc[4] - IQR*1.5
out_max = tri0_stats.iloc[6] + IQR*1.5
tri0_stats = tri0_stats.append(out_min, ignore_index=True)
tri0_stats = tri0_stats.append(out_max, ignore_index=True)

tri0_stats.index = (['count', 'mean', 'std', 'min', '25%', '50%', '75%', 'max', 'range', 'IQR', 'out_min', 'out_max'])

In [30]:
# Calculate IQR / range
print tri0_stats[tri0_stats.index == 'IQR'].x / float(tri0_stats[tri0_stats.index == 'range'].x)
print tri0_stats[tri0_stats.index == 'IQR'].y / float(tri0_stats[tri0_stats.index == 'range'].y)
print tri0_stats[tri0_stats.index == 'IQR'].time / float(tri0_stats[tri0_stats.index == 'range'].time)

IQR    0.002158
Name: x, dtype: float64
IQR    0.100506
Name: y, dtype: float64
IQR    0.593515
Name: time, dtype: float64


In [11]:
# Compute for x
sum(i > out_max.x for i in tri0.x) + sum(i < out_min.x for i in tri0.x) # 68
tri0_bp_rmOLx = tri0[(tri0.x < out_max.x) & (tri0.x > out_min.x)]
tri0_bp_rmOLx.shape # 585

# Compute for y
sum(i > out_max.y for i in tri0.y) + sum(i < out_min.y for i in tri0.y) # 13
tri0_bp_rmOLy = tri0[(tri0.y < out_max.y) & (tri0.y > out_min.y)]
tri0_bp_rmOLy.shape # 640

# Compute for accuracy
sum(i > out_max.accuracy for i in tri0.accuracy) + sum(i < out_min.accuracy for i in tri0.accuracy) # 135
tri0_bp_rmOLacc = tri0[(tri0.accuracy < out_max.accuracy) & (tri0.accuracy > out_min.accuracy)]
tri0_bp_rmOLacc.shape # 518

sum(tri0_bp_rmOLx.row_id.isin(tri0_bp_rmOLy.row_id)) # 582 non-outliers in x and y overlap (89%)
sum(tri0_bp_rmOLx.row_id.isin(tri0_bp_rmOLacc.row_id)) # 482 non-outliers in x and acc overlap (74%)
sum(tri0_bp_rmOLy.row_id.isin(tri0_bp_rmOLacc.row_id)) # 509 non-outliers in y and acc overlap (78%)

tri0_bp_rmOL = tri0_bp_rmOLx[(tri0_bp_rmOLx.y < out_max.y) & (tri0_bp_rmOLx.y > out_min.y)] 

x0_bp = tri0_bp_rmOL.x.mean()
y0_bp = tri0_bp_rmOL.y.mean()
print (x0_bp)
print (y0_bp)

0.795564261168
9.08097216495


In [12]:
# Compute outliers (by stdev)
sum(tri0.x > tri0.x.mean() + tri0.x.std()) + sum(tri0.x < tri0.x.mean() - tri0.x.std()) # 11 only
tri0_std_rmOLx = tri0[(tri0.x < tri0.x.mean() + tri0.x.std()) & (tri0.x > tri0.x.mean() - tri0.x.std())]
print tri0_std_rmOLx.shape # 642

sum(tri0.y > tri0.y.mean() + tri0.y.std()) + sum(tri0.y < tri0.y.mean() - tri0.y.std()) # 145
tri0_std_rmOLy = tri0[(tri0.y < tri0.y.mean() + tri0.y.std()) & (tri0.y > tri0.y.mean() - tri0.y.std())]
print (tri0_std_rmOLy.shape) # 508

tri0_std_rmOLacc = tri0[(tri0.accuracy < tri0.accuracy.mean() + tri0.accuracy.std()) & (tri0.accuracy > tri0.accuracy.mean() - tri0.accuracy.std())]
print (tri0_std_rmOLacc.shape) # 625

# Removed x and y outliers; 507 left
tri0_std_rmOL = tri0_std_rmOLx[(tri0_std_rmOLx.y < tri0.y.mean() + tri0.y.std()) & (tri0_std_rmOLx.y > tri0.y.mean() - tri0.y.std())]

x0_std = tri0_std_rmOL.x.mean()
y0_std = tri0_std_rmOL.y.mean()
print (x0_std)
print (y0_std)


(642, 6)
(508, 6)
(625, 6)
0.798272189349
9.0805122288


In [56]:
tri0_stats.time

import datetime
a = pd.to_datetime(tri0.time, unit = 's')
hr_of_day0 = []
for i in a:
    hr_of_day0.append(
        datetime.datetime.strptime(str(i)[11:], "%H:%M:%S").hour
    )
from collections import Counter
freq = Counter(hr_of_day0)
print (freq.most_common())   # Returns all unique items and their counts
print (freq.most_common(1))  # Returns the highest occurring item
hrHiFreq0 = freq.most_common(1)
hrHiFreq0 = freq.most_common()[0][0] # The hour of day where most check-ins took place
hrLoFreq0 = freq.most_common()[len(freq.most_common())-1][0] # The hour of day where least check-ins took place
hrMean0 = sum(i[1] for i in freq.most_common()) / float(len(freq.most_common())) # Average number of check-ins per h
hrStd0 = (sum((i[1]-hrMean0)**2 for i in freq.most_common()) / float(len(freq.most_common())))**(1/2.0) # How different were the checks of 1 h from another

[(6, 46), (0, 35), (12, 35), (13, 35), (15, 35), (23, 35), (7, 34), (20, 32), (1, 31), (16, 31), (4, 29), (18, 29), (3, 27), (17, 26), (9, 24), (14, 23), (21, 23), (19, 22), (2, 19), (10, 18), (11, 18), (5, 16), (22, 16), (8, 14)]
[(6, 46)]


In [36]:
# location.groupby('place_id')['x'].quantile(0.25)[:3]
#location.groupby('place_id')['x'][]
type(location.groupby('place_id')['x'] > outmax)

ValueError: setting an array element with a sequence.

In [ ]:
location.groupby('place_id')['x'].quantile(0.75)[:3]

In [9]:
f1 = pd.DataFrame()

In [10]:
f1['place_id'] = location.place_id.unique()

In [35]:
f1['IQR_x'] = location.groupby('place_id')['x'].quantile(0.75) - location.groupby('place_id')['x'].quantile(0.25)

In [ ]:
# columns = ['place_id',
#            'IQR_x', 'IQR_y', 'IQR_acc', 'IQR_time',
#            'Range_x', 'Range_y', 'Range_acc', 'Range_time',
#            'IQRRatio_x', 'IQRRatio_y', 'IQRRatio_acc', 'IQRRatio_time',
#            'numOLBP_x', 'numOLBP_y', 'numOLBP_acc', 'numOLBP_time', 
#            'numOLStd_x', 'numOLStd_y', 'numOLStd_acc', 'numOLStd_time', 
#            'meanBP_x', 'stdBP_x', 'meanBP_y', 'stdBP_y',
#            'meanStd_x', 'stdStd_x', 'meanStd_y', 'stdStd_y',
#            'hr_HiFreq', 'hr_LoFreq', 'hr_Mean', 'hr_Std'
#           ]

f1['IQR_y'] = location.groupby('place_id')['y'].quantile(0.75) - location.groupby('place_id')['y'].quantile(0.25)
f1['IQR_acc'] = location.groupby('place_id')['accuracy'].quantile(0.75) - location.groupby('place_id')['accuracy'].quantile(0.25)
f1['IQR_time'] = location.groupby('place_id')['time'].quantile(0.75) - location.groupby('place_id')['time'].quantile(0.25)

f1['Range_x'] = location.groupby('place_id')['x'].max() - location.groupby('place_id')['x'].min()
f1['Range_y'] = location.groupby('place_id')['y'].max() - location.groupby('place_id')['y'].min()
f1['Range_acc'] = location.groupby('place_id')['accuracy'].max() - location.groupby('place_id')['accuracy'].min()
f1['Range_time'] = location.groupby('place_id')['time'].max() - location.groupby('place_id')['time'].min()

f1['IQRRatio_x'] = f1['IQR_x'] / f1['Range_x']
f1['IQRRatio_y'] = location.groupby('place_id')['y'].max() - location.groupby('place_id')['y'].min()
f1['IQRRatio_acc'] = location.groupby('place_id')['accuracy'].max() - location.groupby('place_id')['accuracy'].min()
f1['IQRRatio_time'] = location.groupby('place_id')['time'].max() - location.groupby('place_id')['time'].min()

f1['out_max'] = location.groupby('place_id')['x'].quantile(0.75) + f1['IQR_x']*1.5
f1['out_min'] = location.groupby('place_id')['x'].quantile(0.25) - f1['IQR_x']*1.5
# f1['numOLBP_x'] = location.groupby('place_id')['x'][(location.groupby('place_id')['x'] > location.groupby('place_id')['x'].quantile(0.75) + f1['IQR_x']*1.5)]

In [33]:
# tri0 = location[location.place_id == 1000015801]
# tri1 = location[location.place_id == 1000017288]
# tri2 = location[location.place_id == 1000025138]
# tri0.x.describe()

In [37]:
tri0Stats = getStats(tri0)

In [46]:
import time
t0 = time.time()
tri0.x.loc[tri0['x'] > float(tri0Stats[tri0Stats.index == 'out_max'].x)].count() + tri0.x.loc[tri0['x'] < float(tri0Stats[tri0Stats.index == 'out_min'].x)].count()
t1 = time.time()
print '1st computation took: ', t1 - t0
sum(i > float(tri0Stats[tri0Stats.index == 'out_max'].x) for i in tri0.x) + sum(i < float(tri0Stats[tri0Stats.index == 'out_min'].x) for i in tri0.x)
t2 = time.time()
print '2nd computation took: ', t2 - t1

1st computation took:  0.0025680065155
2nd computation took:  0.0898871421814


In [36]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import datetime
from collections import Counter
import pickle
import time
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KNeighborsClassifier

# path = os.getcwd()
# os.listdir(path)
# location = pd.read_csv(path + '/train.csv')

In [89]:
def getLoc(n, data):
    return data[data.place_id == data.place_id.unique()[n]]

def getStats(locData):
    locDataStats = locData[['x', 'y', 'accuracy', 'time']].describe()
    IQR = locDataStats.iloc[6] - locDataStats.iloc[4]
    rng = locDataStats.iloc[7] - locDataStats.iloc[3]
    locDataStats = locDataStats.append(rng, ignore_index=True)
    locDataStats = locDataStats.append(IQR, ignore_index=True)
    out_min = locDataStats.iloc[4] - IQR*1.5
    out_max = locDataStats.iloc[6] + IQR*1.5
    locDataStats = locDataStats.append(out_min, ignore_index=True)
    locDataStats = locDataStats.append(out_max, ignore_index=True)
    locDataStats.index = (['count', 'mean', 'std', 'min', '25%', '50%', '75%', 'max', 'range', 'IQR', 'out_min', 'out_max'])
    return locDataStats

def getIQR(locDataStats):
    IQR = [locDataStats[locDataStats.index == 'IQR'].x,
           locDataStats[locDataStats.index == 'IQR'].y,
           locDataStats[locDataStats.index == 'IQR'].accuracy,
           locDataStats[locDataStats.index == 'IQR'].time]
    rng = [locDataStats[locDataStats.index == 'range'].x,
           locDataStats[locDataStats.index == 'range'].y,
           locDataStats[locDataStats.index == 'range'].accuracy,
           locDataStats[locDataStats.index == 'range'].time]
    IQRRatio = [locDataStats[locDataStats.index == 'IQR'].x / float(locDataStats[locDataStats.index == 'range'].x),
                locDataStats[locDataStats.index == 'IQR'].y / float(locDataStats[locDataStats.index == 'range'].y),
                locDataStats[locDataStats.index == 'IQR'].accuracy / float(locDataStats[locDataStats.index == 'range'].accuracy),
                locDataStats[locDataStats.index == 'IQR'].time / float(locDataStats[locDataStats.index == 'range'].time)]
    return [IQR, rng, IQRRatio]

def getNumOutliersBP(locDataStats, locData):
#     ol_BPx = sum(i > float(locDataStats[locDataStats.index == 'out_max'].x) for i in locData.x) + sum(i < float(locDataStats[locDataStats.index == 'out_min'].x) for i in locData.x)
#     ol_BPy = sum(i > float(locDataStats[locDataStats.index == 'out_max'].y) for i in locData.y) + sum(i < float(locDataStats[locDataStats.index == 'out_min'].y) for i in locData.y)
#     ol_BPacc = sum(i > float(locDataStats[locDataStats.index == 'out_max'].accuracy) for i in locData.accuracy) + sum(i < float(locDataStats[locDataStats.index == 'out_min'].accuracy) for i in locData.accuracy)
#     ol_BPtime = sum(i > float(locDataStats[locDataStats.index == 'out_max'].time) for i in locData.time) + sum(i < float(locDataStats[locDataStats.index == 'out_min'].time) for i in locData.time)

    ol_BPx = locData.x.loc[locData['x'] > float(locDataStats[locDataStats.index == 'out_max'].x)].count() + locData.x.loc[locData['x'] < float(locDataStats[locDataStats.index == 'out_min'].x)].count()
    ol_BPy = locData.y.loc[locData['y'] > float(locDataStats[locDataStats.index == 'out_max'].y)].count() + locData.y.loc[locData['y'] < float(locDataStats[locDataStats.index == 'out_min'].y)].count()
    ol_BPacc = locData.accuracy.loc[locData['accuracy'] > float(locDataStats[locDataStats.index == 'out_max'].accuracy)].count() + locData.accuracy.loc[locData['accuracy'] < float(locDataStats[locDataStats.index == 'out_min'].accuracy)].count()
    ol_BPtime = locData.time.loc[locData['time'] > float(locDataStats[locDataStats.index == 'out_max'].time)].count() + locData.time.loc[locData['time'] < float(locDataStats[locDataStats.index == 'out_min'].time)].count()

    return [ol_BPx, ol_BPy, ol_BPacc, ol_BPtime]

def getNumOutliersStd(locData):
#     ol_Stdx = sum(locData.x > locData.x.mean() + locData.x.std()) + sum(locData.x < locData.x.mean() - locData.x.std())
#     ol_Stdy = sum(locData.y > locData.y.mean() + locData.y.std()) + sum(locData.y < locData.y.mean() - locData.y.std())
#     ol_Stdacc = sum(locData.accuracy > locData.accuracy.mean() + locData.accuracy.std()) + sum(locData.accuracy < locData.accuracy.mean() - locData.accuracy.std())
#     ol_Stdtime = sum(locData.time > locData.time.mean() + locData.time.std()) + sum(locData.time < locData.time.mean() - locData.time.std())

    ol_Stdx = locData.x.loc[locData.x > locData.x.mean() + locData.x.std()].count() + locData.x.loc[locData.x < locData.x.mean() - locData.x.std()].count()
    ol_Stdy = locData.y.loc[locData.y > locData.y.mean() + locData.y.std()].count() + locData.y.loc[locData.y < locData.y.mean() - locData.y.std()].count()
    ol_Stdacc = locData.accuracy.loc[locData.accuracy > locData.accuracy.mean() + locData.accuracy.std()].count() + locData.accuracy.loc[locData.accuracy < locData.accuracy.mean() - locData.accuracy.std()].count()
    ol_Stdtime = locData.time.loc[locData.time > locData.time.mean() + locData.time.std()].count() + locData.time.loc[locData.time < locData.time.mean() - locData.time.std()].count()

    return [ol_Stdx, ol_Stdy, ol_Stdacc, ol_Stdtime]

def rmOutliersBPxy(locDataStats, locData):
    locData_bp_rmOLx = locData[(locData.x < float(locDataStats[locDataStats.index == 'out_max'].x)) & (locData.x > float(locDataStats[locDataStats.index == 'out_min'].x))]
    locData_bp_rmOL = locData_bp_rmOLx[(locData_bp_rmOLx.y < float(locDataStats[locDataStats.index == 'out_max'].y)) & (locData_bp_rmOLx.y > float(locDataStats[locDataStats.index == 'out_min'].y))]
    return locData_bp_rmOL

def rmOutliersStdxy(locData):
    locData_std_rmOLx = locData[(locData.x < locData.x.mean() + locData.x.std()) & (locData.x > locData.x.mean() - locData.x.std())]
    locData_std_rmOL = locData_std_rmOLx[(locData_std_rmOLx.y < locData.y.mean() + locData.y.std()) & (locData_std_rmOLx.y > locData.y.mean() - locData.y.std())]
    return locData_std_rmOL

def getMeanBPxy(locDataStats, locData):
    locData_rmOLBP = rmOutliersBPxy(locDataStats, locData)    
    return [locData_rmOLBP.x.mean(), locData_rmOLBP.x.std(), locData_rmOLBP.y.mean(), locData_rmOLBP.y.std()]

def getMeanStdxy(locData):
    locData_rmOLStd = rmOutliersStdxy(locData)
    return [locData_rmOLStd.x.mean(), locData_rmOLStd.x.std(), locData_rmOLStd.y.mean(), locData_rmOLStd.y.std()]

def getFreqTime(locData):
    import datetime
    from collections import Counter
    hr_of_day0 = []
    for i in pd.to_datetime(locData.time, unit = 's'):
        hr_of_day0.append(
            datetime.datetime.strptime(str(i)[11:], "%H:%M:%S").hour
        )
    freq = Counter(hr_of_day0)
    hrHiFreq = freq.most_common()[0][0] # The hour of day where most check-ins took place
    hrLoFreq = freq.most_common()[len(freq.most_common())-1][0] # The hour of day where least check-ins took place
    hrMean = sum(i[1] for i in freq.most_common()) / float(len(freq.most_common())) # Average number of check-ins per h
    hrStd = (sum((i[1]-hrMean)**2 for i in freq.most_common()) / float(len(freq.most_common())))**(1/2.0) # How different were the checks of 1 h from another
    return [hrHiFreq, hrLoFreq, hrMean, hrStd]

def getFreqDay(locData):
    day_of_wk0 = [i.dayofweek for i in pd.to_datetime(locData.time, unit = 's')]
    freq = Counter(day_of_wk0)
    wkHiFreq = freq.most_common()[0][0] # The hour of day where most check-ins took place
    wkLoFreq = freq.most_common()[len(freq.most_common())-1][0] # The hour of day where least check-ins took place
    is_wkend = np.where((np.asarray(day_of_wk0)==5) | (np.asarray(day_of_wk0)==6), 1, 0)
    not_wkend = np.where((np.asarray(day_of_wk0)==5) | (np.asarray(day_of_wk0)==6), 0, 1)
    # wkendRatio = is_wkend.sum() / float(len(day_of_wk0))
    wkendRatio = (is_wkend.sum()/2.0) / (not_wkend.sum()/5.0)
    return [wkHiFreq, wkLoFreq, wkendRatio]


In [81]:
# tmpAll = []
for i in range(len(location.place_id.unique()))[91073:]:
    tmp = []
    tmp.append(int(location.place_id.unique()[i])) # len(tmp) = 1
    loc = getLoc(n = i, data = location)
    locStats = getStats(loc)
    
    for m in getIQR(locStats): # 3 lists of 4 elements each.
        for n in m: # 4 elements
            tmp.append(n.values[0]) # len(tmp) = 13

    tmp.extend(getNumOutliersBP(locStats, loc)) # len(tmp) = 17
    tmp.extend(getNumOutliersStd(loc)) # len(tmp) = 21

    tmp.extend(getMeanBPxy(locStats, loc)) # len(tmp) = 25
    tmp.extend(getMeanStdxy(loc)) # len(tmp) = 29
    
    tmp.extend(getFreqTime(loc)) # len(tmp) = 33
    tmpAll.append(tmp)
    if i % 100 == 0:
        print i
        print 'Number of data points collected: ', len(tmpAll)
 

91100
Number of data points collected:  52
91200
Number of data points collected:  152
91300
Number of data points collected:  252
91400
Number of data points collected:  352
91500
Number of data points collected:  452
91600
Number of data points collected:  552
91700
Number of data points collected:  652
91800
Number of data points collected:  752
91900
Number of data points collected:  852
92000
Number of data points collected:  952
92100
Number of data points collected:  1052
92200
Number of data points collected:  1152
92300
Number of data points collected:  1252
92400
Number of data points collected:  1352
92500
Number of data points collected:  1452
92600
Number of data points collected:  1552
92700
Number of data points collected:  1652
92800
Number of data points collected:  1752
92900
Number of data points collected:  1852
93000
Number of data points collected:  1952
93100
Number of data points collected:  2052
93200
Number of data points collected:  2152
93300
Number of data 

In [83]:
columns = ['place_id',
           'IQR_x', 'IQR_y', 'IQR_acc', 'IQR_time',
           'Range_x', 'Range_y', 'Range_acc', 'Range_time',
           'IQRRatio_x', 'IQRRatio_y', 'IQRRatio_acc', 'IQRRatio_time',
           'numOLBP_x', 'numOLBP_y', 'numOLBP_acc', 'numOLBP_time', 
           'numOLStd_x', 'numOLStd_y', 'numOLStd_acc', 'numOLStd_time', 
           'meanBP_x', 'stdBP_x', 'meanBP_y', 'stdBP_y',
           'meanStd_x', 'stdStd_x', 'meanStd_y', 'stdStd_y',
           'hr_HiFreq', 'hr_LoFreq', 'hr_Mean', 'hr_Std'
          ]

features4 = pd.DataFrame(tmpAll, columns = columns)
print features4.shape
pickle.dump(features4, open('features4.p', 'wb'))

(17341, 33)


In [90]:
tmpAll = []
for i in range(len(location.place_id.unique())):
    tmp = []
    loc = getLoc(n = i, data = location)
    tmp.extend(getFreqDay(loc))
    tmpAll.append(tmp)
    if i % 100 == 0:
        print i
        print 'Number of data points collected: ', len(tmpAll)

0
Number of data points collected:  1
100
Number of data points collected:  101
200
Number of data points collected:  201
300
Number of data points collected:  301
400
Number of data points collected:  401
500
Number of data points collected:  501
600
Number of data points collected:  601
700
Number of data points collected:  701
800
Number of data points collected:  801
900
Number of data points collected:  901
1000
Number of data points collected:  1001
1100
Number of data points collected:  1101
1200
Number of data points collected:  1201
1300
Number of data points collected:  1301
1400
Number of data points collected:  1401
1500
Number of data points collected:  1501
1600
Number of data points collected:  1601
1700
Number of data points collected:  1701
1800
Number of data points collected:  1801
1900
Number of data points collected:  1901
2000
Number of data points collected:  2001
2100
Number of data points collected:  2101
2200
Number of data points collected:  2201
2300
Number 

KeyboardInterrupt: 

In [98]:
print i
print len(tmpAll)

10247
10247


In [99]:
columns_pt2 = ['wkHiFreq', 'wkLoFreq', 'wkendRatio']
features_pt2 = pd.DataFrame(tmpAll, columns = columns_pt2)
print features_pt2.shape
pickle.dump(features_pt2, open('features_pt2.p', 'wb'))

(10247, 3)


In [ ]:
# Use groupby to get features

In [142]:
locSmall = location.iloc[:10000, :]

locSmall['timeFmtted'] = pd.to_datetime(locSmall.time, unit = 'm')

extractHr = lambda x: datetime.datetime.strptime(str(x)[11:], "%H:%M:%S").hour
locSmall['hour'] = locSmall.timeFmtted.apply(extractHr)

extractDay = lambda x: x.dayofweek
locSmall['weekday'] = locSmall.timeFmtted.apply(extractDay)

locSmall['isWeekend'] = np.where((locSmall['weekday']==5) | (locSmall['weekday']==6), 1, 0)

/Users/yingjiang/miniconda2/lib/python2.7/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [147]:
locSmallG = pd.DataFrame()
locSmallG['hrHiFreq'] = locSmall.groupby('place_id')['hour'].agg(lambda x: x.value_counts().index[0])
locSmallG['hrLoFreq'] = locSmall.groupby('place_id')['hour'].agg(lambda x: x.value_counts().index[-1])
locSmallG['dayHiFreq'] = locSmall.groupby('place_id')['weekday'].agg(lambda x: x.value_counts().index[0])
locSmallG['dayLoFreq'] = locSmall.groupby('place_id')['weekday'].agg(lambda x: x.value_counts().index[-1])
locSmallG['wkendRatio'] = locSmall.groupby('place_id')['isWeekend'].mean()



In [149]:
# Groupby scale up

location['timeFmtted'] = pd.to_datetime(location.time, unit = 'm')

extractHr = lambda x: datetime.datetime.strptime(str(x)[11:], "%H:%M:%S").hour
location['hour'] = location.timeFmtted.apply(extractHr)

extractDay = lambda x: x.dayofweek
location['weekday'] = location.timeFmtted.apply(extractDay)

location['isWeekend'] = np.where((location['weekday']==5) | (location['weekday']==6), 1, 0)

In [150]:
locationG = pd.DataFrame()
locationG['hrHiFreq'] = location.groupby('place_id')['hour'].agg(lambda x: x.value_counts().index[0])
locationG['hrLoFreq'] = location.groupby('place_id')['hour'].agg(lambda x: x.value_counts().index[-1])
locationG['dayHiFreq'] = location.groupby('place_id')['weekday'].agg(lambda x: x.value_counts().index[0])
locationG['dayLoFreq'] = location.groupby('place_id')['weekday'].agg(lambda x: x.value_counts().index[-1])
locationG['wkendRatio'] = location.groupby('place_id')['isWeekend'].mean()
locationG['place_id'] = locationG.index

In [169]:
print locationG.shape
pickle.dump(locationG, open('features_time.p', 'wb'))

(108390, 6)


In [86]:
path = '/Users/yingjiang/Dropbox/Learnings/Stats_data/Projects/FBCheckins'
os.chdir(path)
features = pickle.load(open('features.p', 'rb'))
features0 = pickle.load(open('features0.p', 'rb'))
features1 = pickle.load(open('features1.p', 'rb'))
features2 = pickle.load(open('features2.p', 'rb'))
features3 = pickle.load(open('features3.p', 'rb'))
features4 = pickle.load(open('features4.p', 'rb'))
features = pd.concat([features, features0, features1, features2, features3, features4])

features = features.reset_index(range(features.shape[0]))
pickle.dump(features, open('features_loc.p', 'wb'))


In [193]:
featuresAll = pd.merge(features, locationG, on = 'place_id', how = 'inner')
pickle.dump(featuresAll, open('features_all.p', 'wb'))

In [ ]:
# Predict

In [62]:
trainX = features[['meanStd_x', 'meanStd_y']]
trainY = features.place_id
print trainX.shape
print trainY.shape

print trainX.meanStd_x.isnull().sum()
print trainX.meanStd_y.isnull().sum()
print trainX.meanStd_x.index[trainX.meanStd_x.apply(np.isnan)]
print trainX.meanStd_y.index[trainX.meanStd_y.apply(np.isnan)]

trainX = trainX.dropna()
trainY = trainY.drop(trainY.index[[89497]])
print trainX.shape
print trainY.shape

(91073, 2)
(91073,)


In [215]:
featuresAll = pickle.load(open('features_all.p', 'rb'))
featuresAll = featuresAll.dropna()
trainX_time = featuresAll[['meanStd_x', 'meanStd_y', 'hrHiFreq', 'dayHiFreq']]
# trainX_time = trainX_time.dropna()
trainY_time = featuresAll.place_id
print trainX_time.shape
print trainY_time.shape

(108018, 4)
(108018,)


In [194]:
test = pd.read_csv(path + '/test.csv')
testX = test[['x', 'y']]
print test.shape

(8607230, 5)


In [210]:
# testX_time = test[['x', 'y']]
# testX_time['time'] = pd.to_datetime(test.time, unit = 'm')
# testX_time['hour'] = testX_time.time.apply(lambda x: datetime.datetime.strptime(str(x)[11:], "%H:%M:%S").hour)
# testX_time['weekday'] = testX_time.time.apply(lambda x: x.dayofweek)
# testX_time['hour'] = testX_time.time.apply(extractHr)
# testX_time['weekday'] = testX_time.time.apply(extractDay)

testX_time = testX_time.drop('time', axis = 1)
print testX_time.shape

(8607230, 4)


In [211]:
testX_time.head()

,x,y,hour,weekday
0,0.1675,1.3608,10,5
1,7.3909,2.5301,3,0
2,8.0978,2.3473,10,2
3,0.9990,1.0591,1,3
4,0.6670,9.7254,23,0


In [216]:
knn = KNeighborsClassifier(n_neighbors = 1)
knn.fit(trainX_time, trainY_time) 
testY = knn.predict(testX_time)

In [217]:
len(testY)

8607230

In [218]:
testY2 = pd.DataFrame(
    {
        "place_id": testY
                   
}
    )

In [219]:
testY2.to_csv('FBCheckin_test_final.csv')